In [1]:
import requests
import time
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
options = Options()
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)

In [3]:
categories = [
    'https://www.amazon.ca/Best-Sellers-Music/zgbs/music/',
    'https://www.amazon.ca/Best-Sellers-DVD/zgbs/dvd/',
    'https://www.amazon.ca/Best-Sellers-Electronics/zgbs/electronics/'
]

In [4]:
categories

['https://www.amazon.ca/Best-Sellers-Music/zgbs/music/',
 'https://www.amazon.ca/Best-Sellers-DVD/zgbs/dvd/',
 'https://www.amazon.ca/Best-Sellers-Electronics/zgbs/electronics/']

In [5]:
first_url = categories[2]

In [6]:
driver.get(first_url)

KeyboardInterrupt: 

In [ ]:
body_el = driver.find_element_by_css_selector('body')
html_str = body_el.get_attribute('innerHTML')

In [ ]:
html_obj = HTML(html=html_str)

In [ ]:
new_links = [x for x in html_obj.links if x.startswith('/')]
new_links = [x for x in new_links if 'product-reviews/' not in x]

In [ ]:
page_links = [f'https://www.amazon.ca{x}' for x in new_links]

In [ ]:
# page_links

In [ ]:
def scrape_product_page(url, title_lookup = '#productTitle', price_lookup = '#priceblock_ourprice'):
    driver.get(url)
    time.sleep(1.2)
    body_el = driver.find_element_by_css_selector('body')
    html_str = body_el.get_attribute('innerHTML')
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price


In [ ]:
first_product_link = page_links[0]
first_product_link

In [ ]:
for link in page_links:
    title, price = (None, None)
    try:
        title, price = scrape_product_page(link)
    except:
        pass
    if title != None and price != None:
        print(link, title, price)